# Necessary libraries

In [1]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.subplots as sp
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics.pairwise import haversine_distances
from math import radians
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans,DBSCAN
from sklearn.metrics import silhouette_score


# EDA on April 2014

In [2]:
avril = pd.read_csv('data/uber-raw-data-apr14.csv')
avril.shape
# Huge amount of data, will need to reduce dataset later

(564516, 4)

In [3]:
avril.head()

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512


In [4]:
# Convert time into day of the week and hour
avril['Date'] =  pd.to_datetime(avril['Date/Time'])
avril['Day_of_week'] = avril.Date.dt.day_name()
avril['Hour'] = avril.Date.dt.hour
# Remove original datetime and useless column Base
avril = avril.drop(columns=['Date/Time','Date','Base'])
avril.head()
# Create column for visualization
avril['Type'] = avril['Day_of_week'].apply(lambda x: 'Friday' if x == 'Friday' else 'Saturday' if x == 'Saturday' else 'Sunday' if x == 'Sunday' else 'Weekday')

In [5]:
avril.head()

,Lat,Lon,Day_of_week,Hour,Type
0,40.7690,-73.9549,Tuesday,0,Weekday
1,40.7267,-74.0345,Tuesday,0,Weekday
2,40.7316,-73.9873,Tuesday,0,Weekday
3,40.7588,-73.9776,Tuesday,0,Weekday
4,40.7594,-73.9722,Tuesday,0,Weekday


In [6]:
avril.describe(include='all')

,Lat,Lon,Day_of_week,Hour,Type
count,564516.000000,564516.000000,564516,564516.000000,564516
unique,NaN,NaN,7,NaN,4
top,NaN,NaN,Wednesday,NaN,Weekday
freq,NaN,NaN,108631,NaN,345744
mean,40.740005,-73.976817,NaN,14.465043,NaN
std,0.036083,0.050426,NaN,5.873925,NaN
min,40.072900,-74.773300,NaN,0.000000,NaN
25%,40.722500,-73.997700,NaN,10.000000,NaN
50%,40.742500,-73.984800,NaN,16.000000,NaN
75%,40.760700,-73.970000,NaN,19.000000,NaN


In [7]:
display(100*avril.isnull().sum()/avril.shape[0])
# No missing value

Lat            0.0
Lon            0.0
Day_of_week    0.0
Hour           0.0
Type           0.0
dtype: float64

In [8]:
# Sort the dataset
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
avril['Day_of_week'] = pd.Categorical(avril['Day_of_week'], categories=day_order, ordered=True)
avril = avril.sort_values(['Day_of_week', 'Hour'])
avril.head()

,Lat,Lon,Day_of_week,Hour,Type
7785,40.7205,-73.9939,Monday,0,Weekday
7786,40.7407,-74.0077,Monday,0,Weekday
7787,40.7591,-73.9892,Monday,0,Weekday
7788,40.7419,-74.0034,Monday,0,Weekday
15857,40.7456,-73.9773,Monday,0,Weekday


In [9]:
# Uber demands distribution along the week
histogram = go.Histogram(x=avril['Day_of_week'])
layout = go.Layout(title='Distribution of Uber Demands over the Week')
fig = go.Figure(data=[histogram], layout=layout)
# fig.show()
print("We already observe different demands depending on the day of the week.")

We already observe different demands depending on the day of the week.


![alt text](https://raw.githubusercontent.com/elodiesune/PROJECT_Uber/main/pictures/1_Distribution_of_uber_pickups_on_the_week.jpg)

In [10]:
# Demands depending on time and days

# Separate data into bins
bins = [0, 5, 11, 14, 16, 20, 23]
labels = ['Early_morning', 'Morning', 'Lunch', 'Work', 'Afterwork', 'Early_night']
avril['Hour_bin'] = pd.cut(avril['Hour'], bins=bins, labels=labels, right=False)

# Get the hour bins for each 'Type'
weekday_bins = avril.loc[avril['Type'] == 'Weekday', 'Hour_bin']
friday_bins = avril.loc[avril['Type'] == 'Friday', 'Hour_bin']
saturday_bins = avril.loc[avril['Type'] == 'Saturday', 'Hour_bin']
sunday_bins = avril.loc[avril['Type'] == 'Sunday', 'Hour_bin']

# Get values for each bins
weekday_values = weekday_bins.value_counts(sort=False)
friday_values = friday_bins.value_counts(sort=False)
saturday_values = saturday_bins.value_counts(sort=False)
sunday_values = sunday_bins.value_counts(sort=False)

# Pies
weekday_pie = go.Pie(labels=labels,values=weekday_values,textinfo='label')
friday_pie = go.Pie(labels=labels,values=friday_values,textinfo='label')
saturday_pie = go.Pie(labels=labels,values=saturday_values,textinfo='label')
sunday_pie = go.Pie(labels=labels,values=sunday_values,textinfo='label')

# Plot
fig = make_subplots(rows=1,cols=4,subplot_titles=['Weekday', 'Friday', 'Saturday', 'Sunday'],specs=[[{'type': 'pie'}, {'type': 'pie'}, {'type': 'pie'}, {'type': 'pie'}]],horizontal_spacing=0.05,)
fig.add_trace(weekday_pie, row=1, col=1)
fig.add_trace(friday_pie, row=1, col=2)
fig.add_trace(saturday_pie, row=1, col=3)
fig.add_trace(sunday_pie, row=1, col=4)
fig.update_layout(showlegend=False,title='Uber demands by time',title_x=0.5,title_font=dict(size=20),height=400,width=1000)
# fig.show()

print("Except for the afterwork time, which is always very demanded, we can see clear differences in uber demands along the week.")
print("People ask for uber in morning only during the weekdays and fridays.")
print("People party on friday and saturday evenings, which make more early_night demands than in weekdays and sundays.")
print("Obviously for people coming back late, it increases early_morning demands on saturdays and sundays.")
print("There are much more demand at lunch time during sundays.")

Except for the afterwork time, which is always very demanded, we can see clear differences in uber demands along the week.
People ask for uber in morning only during the weekdays and fridays.
People party on friday and saturday evenings, which make more early_night demands than in weekdays and sundays.
Obviously for people coming back late, it increases early_morning demands on saturdays and sundays.
There are much more demand at lunch time during sundays.


![alt text](https://raw.githubusercontent.com/elodiesune/PROJECT_Uber/main/pictures/2_Uber_demands_by_time.jpg)

In [41]:
# General map of uber demands in April

fig = px.scatter_mapbox(title="Uber demands in NYC", data_frame=avril,lat='Lat',lon='Lon',color='Hour',mapbox_style='open-street-map')
# fig.show(renderer='notebook')
print("We already see the crazy high density of uber demands in the center of NY (Manhattan)")

We already see the crazy high density of uber demands in the center of NY (Manhattan)


![alt text](https://raw.githubusercontent.com/elodiesune/PROJECT_Uber/main/pictures/3_Uber_demands_by_time.jpg)

# Machine Learning on days

## KMeans

#### Preprocessing

In [12]:
# Before studying each day, we will make a quick study on days from monday to friday vs saturday-sunday.
# This should help us deciding if we can apply the same number of clusters for each day later. 

In [13]:
# Create one dataframe for weekend/weekday ; keep only latitude and longitude variables
weekend_mask = (avril['Type'].isin(['Saturday', 'Sunday']))
weekday_mask = (avril['Type'].isin(['Weekday', 'Friday']))
avril_weekend = avril.loc[weekend_mask, ['Lat','Lon']].sample(10000)
avril_weekday = avril.loc[weekday_mask, ['Lat','Lon']].sample(10000)

In [14]:
# GPS coordinates do not follow a gaussian distribution ==> we should not use standard scaler.
# Most common methods : usage of a distance metric suitable for spherical coordinates (instead of the Euclidean distance)

# Convert latitude and longitude to radians
X1 = avril_weekend[['Lat', 'Lon']].applymap(radians)
X2 = avril_weekday[['Lat', 'Lon']].applymap(radians)

# Calculate Haversine distances between coordinates (in kilometers)
X1_processed = haversine_distances(X1)
X2_processed = haversine_distances(X2)

#### Finding optimal number of clusters

In [15]:
# Instanciate KMeans

# Instantiate with k=4 and initialisation with k-means
kmeans = KMeans(n_clusters=4, random_state=0)

kmeans1 = kmeans
kmeans2 = kmeans

# Fit kmeans to our dataset
kmeans1.fit(X1_processed)
kmeans2.fit(X2_processed)

KMeans(n_clusters=4, random_state=0)

In [16]:
%%time
# Calculate inertia and silhouette scores for both dataframes

# Weekends
wcss1 =  []
k1 = []
sil1 = []

for i in range (2,40): 
    kmeans1 = KMeans(n_clusters= i, random_state = 0)
    kmeans1.fit(X1_processed)
    wcss1.append(kmeans1.inertia_)
    sil1.append(silhouette_score(X1_processed, kmeans1.predict(X1_processed)))
    k1.append(i)
    # print("WCSS for K={} --> {}".format(i, wcss1[-1]))

# Weekdays
wcss2 =  []
k2 = []
sil2 = []

for i in range (2,40): 
    kmeans2 = KMeans(n_clusters= i, random_state = 0)
    kmeans2.fit(X2_processed)
    wcss2.append(kmeans2.inertia_)
    sil2.append(silhouette_score(X2_processed, kmeans2.predict(X2_processed)))
    k2.append(i)
    # print("WCSS for K={} --> {}".format(i, wcss2[-1]))

Wall time: 1h 16s


In [ ]:
# Plot inertia with subplot

fig = sp.make_subplots(rows=1, cols=2, subplot_titles=("Weekends", "Weekdays"))

# Weekends
wcss_frame1 = pd.DataFrame(wcss1)
k_frame1= pd.Series(k1)
fig.add_trace(go.Scatter(x=k_frame1, y=wcss_frame1.iloc[:,-1], mode='lines'), row=1, col=1)

# Weekdays
wcss_frame2 = pd.DataFrame(wcss2)
k_frame2= pd.Series(k2)
fig.add_trace(go.Scatter(x=k_frame2, y=wcss_frame2.iloc[:,-1], mode='lines'), row=1, col=2)

fig.update_layout(height=400, width=800, showlegend=False, title = 'Inertia per number of clusters')
fig.update_xaxes(title_text="Clusters", row=2, col=1)
fig.update_yaxes(title_text="Inertia", row=1, col=1)
fig.update_yaxes(title_text="Inertia", row=2, col=1)

# fig.show()

![alt text](https://raw.githubusercontent.com/elodiesune/PROJECT_Uber/main/pictures/4_KMeans_trial_inertia.jpg)

In [18]:
print("We want to minimize inertia as a first objective : it would mean that GPS coordinates within each cluster are more concentrated, so the distances would shorter. Thus, the driving time too (excluding traffic problems).")
print("Between 10 and 20 k seems a good compromize (we almost reach a plateau) without taking too much time.")

We want to minimize inertia as a first objective : it would mean that GPS coordinates within each cluster are more concentrated, so the distances would shorter. Thus, the driving time too (excluding traffic problems).
Between 10 and 20 k seems a good compromize (we almost reach a plateau) without taking too much time.


In [ ]:
# Plot silhouette score with subplot

fig = sp.make_subplots(rows=1, cols=2, subplot_titles=("Weekends", "Weekdays"))

# Weekends
cluster_scores1=pd.DataFrame(sil1)
k_frame1 = pd.Series(k1)
fig.add_trace(go.Bar(x=k_frame1, y=cluster_scores1.iloc[:, -1]), row=1, col=1)

# Weekdays
cluster_scores2=pd.DataFrame(sil2)
k_frame2 = pd.Series(k2)
fig.add_trace(go.Bar(x=k_frame2, y=cluster_scores2.iloc[:, -1]), row=1, col=2)

fig.update_layout(height=500, width=900, showlegend=False, title = "Silhouette score per number of clusters")
fig.update_xaxes(title_text="Clusters", row=1, col=1)
fig.update_xaxes(title_text="Clusters", row=1, col=2)
fig.update_yaxes(title_text="Silhouette Score", row=1, col=1)

# fig.show()

![alt text](https://raw.githubusercontent.com/elodiesune/PROJECT_Uber/main/pictures/5_KMeans_trial_silhouette.jpg)

In [43]:
print("To lower inertia, we want 10 to 20 clusters (end of the elbow).")
print("Now, we want silhouette score as high as possible as it means well defined and well separated clusters.")
print("For weekends, the highest silhouette score (between 10 and 20 clusters) is at k = 13")
print("For weekdays, the highest silhouette score (between 10 and 20 clusters) is at k = 15")

To lower inertia, we want 10 to 20 clusters (end of the elbow).
Now, we want silhouette score as high as possible as it means well defined and well separated clusters.
For weekends, the highest silhouette score (between 10 and 20 clusters) is at k = 13
For weekdays, the highest silhouette score (between 10 and 20 clusters) is at k = 15


In [21]:
print('This shows that for further analysis, we will need to design a number of cluster adapted to each day.')

This shows that for further analysis, we will need to design a number of cluster adapted to each day.


#### Running the model with optimal number of clusters

In [44]:
%%time

kmeans = KMeans(n_clusters= 13)
kmeans.fit(X1_processed)
avril_weekend.loc[:,'Cluster_KMeans'] = kmeans.predict(X1_processed)

kmeans = KMeans(n_clusters= 15)
kmeans.fit(X2_processed)
avril_weekday.loc[:,'Cluster_KMeans'] = kmeans.predict(X2_processed)

Wall time: 32.7 s


In [45]:
# Final dataframe

avril_weekend["Type"] = "Weekend"
avril_weekday["Type"] = "Weekday"

final_df_kmeans_trial = pd.concat([avril_weekend, avril_weekday], ignore_index=True)
final_df_kmeans_trial.head()

,Lat,Lon,Cluster_KMeans,Type
0,40.7611,-73.9676,8,Weekend
1,40.7771,-73.9560,2,Weekend
2,40.7533,-73.9703,8,Weekend
3,40.7392,-74.0029,3,Weekend
4,40.7631,-73.9675,8,Weekend


In [ ]:
# Map KMeans on interactive graph

fig = px.scatter_mapbox(width=500,height=500, title = 'Clusters by KMeans - Weekends vs Weekdays', data_frame=final_df_kmeans_trial,lat='Lat',lon='Lon',color=final_df_kmeans_trial['Cluster_KMeans'].astype('str'),mapbox_style='open-street-map',color_continuous_scale='turbo',animation_frame = "Type")
# fig.show()

![alt text](https://raw.githubusercontent.com/elodiesune/PROJECT_Uber/main/pictures/6_KMeans_trial_clusters.jpg)

In [27]:
print("We see that the general method is working, and that clusters are already different in weekends vs weekdays.")

We see that the general method is working, and that clusters are already different in weekends vs weekdays.


#### Loop for each day of the week

In [28]:
# Create one dataframe per day

days_of_week = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
dataframes = []
    
for day in days_of_week:
    mask = (avril['Day_of_week'] == day)
    avril_day = avril.loc[mask,['Lat','Lon']].sample(n=10000,replace=True)
    dataframes.append(avril_day)

# Process data for each dataframe

X_processed_list = []

for df in dataframes:
    X = df[['Lat', 'Lon']].applymap(radians)
    X_processed = haversine_distances(X)
    X_processed_list.append(X_processed)

In [29]:
%%time

# Find best k for each dataframe

best_k_list = []

for i, X_processed in enumerate(X_processed_list):
    df = dataframes[i].copy()
    silhouette_score_list = []
    # we know that the end of the elbow is generally between 10 and 20.
    # with more time and powerful machines, we could test more k to be more accurate
    for k in range(10,20): 
        kmeans = KMeans(n_clusters=k, random_state=0)
        kmeans.fit(X_processed)
        silhouette_score_list.append(silhouette_score(X_processed, kmeans.predict(X_processed)))
    best_silhouette_score = max(silhouette_score_list) # we want the highest silhouette score at the end of the elbow
    best_k = 10 + silhouette_score_list.index(best_silhouette_score)
    best_k_list.append(best_k) # best k for each dataframe

Wall time: 59min 14s


In [30]:
print(best_k_list)
print("We see different numbers of clusters depending on the day.")

[16, 11, 14, 11, 16, 13, 15]
We see different numbers of clusters depending on the day.


In [31]:
# Apply best k to each dataframe and save predictions

final_df_kmeans = pd.DataFrame(columns=['Lat','Lon','Cluster_KMeans','Day_of_week'])

for i, X_processed in enumerate(X_processed_list):
    df = dataframes[i].copy()
    k = best_k_list[i]
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(X_processed)
    df['Cluster_KMeans'] = kmeans.predict(X_processed)
    df['Day_of_week'] = days_of_week[i]
    final_df_kmeans = pd.concat([final_df_kmeans, df], ignore_index=True)

In [ ]:
fig = px.scatter_mapbox(width = 800, title = 'Cluster by KMeans per day of week', data_frame=final_df_kmeans,lat='Lat',lon='Lon',color=final_df_kmeans['Cluster_KMeans'].astype('str'),mapbox_style='open-street-map',color_continuous_scale='turbo',animation_frame = "Day_of_week")
# fig.show()
fig.write_html("pictures/kmeans_animation.html")

![alt text](https://raw.githubusercontent.com/elodiesune/PROJECT_Uber/main/pictures/7_KMeans_final_clusters_by_day.jpg)

In [51]:
print("The clusters are shown in the order by descending density. The most important ones (in the center) are quite small.")
print("In comparison with google map, it seems that the distance in the central clusters is around 10min by car, which is what we wanted.")
print("However, we see that kmeans here is not efficient for less dense zones, and group points which are very far for each others.")
print("Taxi drivers should focus on dense zones anywway, so we could just decide to ignore those zones.")

The clusters are shown in the order by descending density. The most important ones (in the center) are quite small.
In comparison with google map, it seems that the distance in the central clusters is around 10min by car, which is what we wanted.
However, we see that kmeans here is not efficient for less dense zones, and group points which are very far for each others.
Taxi drivers should focus on dense zones anywway, so we could just decide to ignore those zones.


## DBSCAN

### Trial on a sample (all days mixed)

In [ ]:
%%time

# Same preprocessing for  DBSCAN

X = avril.copy().sample(10000)
X = X.iloc[:,:2]
X_processed = X[['Lat', 'Lon']].applymap(radians)
X_processed = haversine_distances(X_processed)

# DBSCAN
# epsilon: radius within which neighboring points are considered part of the same cluster (density).
# min_samples: minimum number of points required to form a dense region (core point)
# Ball-tree algorithm: used for non euclidian distances

db = DBSCAN(eps=0.005, min_samples=10, algorithm="ball_tree") # instantiate
db.fit(X_processed) # fit
X['Cluster_DBSCAN'] = db.labels_ # get predictions (clusters)

# Create the scatter mapbox plot
fig = px.scatter_mapbox(X, lat='Lat', lon='Lon', width = 800, color=X['Cluster_DBSCAN'].astype('str'), hover_data=['Cluster_DBSCAN'], mapbox_style='open-street-map')
fig.update_layout(title='DBSCAN Clustering (all days mixed test)')
# fig.show()

![alt text](https://raw.githubusercontent.com/elodiesune/PROJECT_Uber/main/pictures/8_DBSCAN_trial.jpg)

### Loop on every day of the week

In [36]:
%%time

# We create the final dataframe containing clusters
final_df_dbscan = pd.DataFrame(columns=['Lat','Lon','Cluster_DBSCAN','Day_of_week'])

# We use the variables "dataframes" and "X_processed_list" created for each day previously
for i, X_processed in enumerate(X_processed_list):
    df = dataframes[i].copy()
    db = DBSCAN(eps=0.005, min_samples=7, algorithm="ball_tree") # instantiate
    db.fit(X_processed) # fit
    df['Cluster_DBSCAN'] = db.labels_ # get predictions (clusters)
    df['Day_of_week'] = days_of_week[i]
    final_df_dbscan = pd.concat([final_df_dbscan, df], ignore_index=True)

Wall time: 14min 43s


In [37]:
# Count number of outliers (cluster -1)
outliers = final_df_dbscan[final_df_dbscan['Cluster_DBSCAN'] == -1]
percentage_outliers = len(outliers) * 100 / len(final_df_dbscan)
print(f"Percentage of outliers: {round(percentage_outliers)}%, we can remove them.")

# Remove them
final_df_dbscan = final_df_dbscan.drop(outliers.index)

Percentage of outliers: 4%


In [38]:
print(final_df_dbscan.groupby('Day_of_week').Cluster_DBSCAN.nunique())
print("We see here that the number of clusters can vary a lot depending on the day.")

Day_of_week
Friday       16
Monday       22
Saturday     15
Sunday       20
Thursday     17
Tuesday      21
Wednesday    17
Name: Cluster_DBSCAN, dtype: int64
We see here that the number of clusters can vary a lot depending on the day.


In [ ]:
fig = px.scatter_mapbox(title = 'Cluster by DBScan per day of week', width = 800, data_frame=final_df_dbscan,lat='Lat',lon='Lon',color=final_df_dbscan['Cluster_DBSCAN'].astype('str'),mapbox_style='open-street-map',color_continuous_scale='turbo',animation_frame = "Day_of_week")
# fig.show()
fig.write_html("pictures/dbscan_animation.html")

![alt text](https://raw.githubusercontent.com/elodiesune/PROJECT_Uber/main/pictures/9_DBSCAN_final_clusters_by_day.jpg)

In [56]:
print("DBSCAN seems very good for finding hotspots. Removal of outliers makes clustering much better than with KMeans.")
print("It eliminates the problem of not dense but very long zones that we had with KMeans.")
print("We still have the central zone, which should be a bit smaller if we want drivers to be always less than 10min from a demand.")

DBSCAN seems very good for finding hotspots. Removal of outliers makes clustering much better than with KMeans.
It eliminates the problem of not dense but very long zones that we had with KMeans.
We still have the central zone, which should be a bit smaller if we want drivers to be always less than 10min from a demand.


# CONCLUSION

##### 

The best method for our problem is DBSCAN, for the following reasons:
      - unknown number of clusters for each day and hour
      - many outliers far around New York city
      - various densities depending on the zone in the city
      - irregular shapes of clusters

However, the central zone around Manhattan (the densest one) should be clustered more, in order to respect the time asked (less than 10min driving). Or another solution would be to use KMEANS which had very nice little clusters, but remove outliers before. 

To pursue the analysis per hour, we would need to separate the file in 168 dataframes (7 days, 24 hours) and do the exact same process as we did previously. The method to use would be the DBSCAN, as said before, because of unknown number of clusters, numerous outliers and irregular densities in New York City. 